In [138]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import polars as pl
import pyarrow as pa

# Creating Player x Tourn. x Year Panel

## 1. Data Prep

In [139]:
# Load the combines matches dataset (2000 - 2024 Mens Singles)
df = pd.read_csv(r"C:\Users\shivg\OneDrive\Desktop\GitHub\Tennis_ATP\data\all_matches.csv", low_memory=False)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110397 entries, 0 to 110396
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          110397 non-null  int64  
 1   tourney_id          110397 non-null  object 
 2   tourney_name        110397 non-null  object 
 3   surface             110344 non-null  object 
 4   draw_size           110397 non-null  int64  
 5   tourney_level       110397 non-null  object 
 6   tourney_date        110397 non-null  int64  
 7   match_num           110397 non-null  int64  
 8   winner_id           110397 non-null  int64  
 9   winner_seed         44673 non-null   object 
 10  winner_entry        13526 non-null   object 
 11  winner_name         110397 non-null  object 
 12  winner_hand         110390 non-null  object 
 13  winner_ht           107746 non-null  float64
 14  winner_ioc          110397 non-null  object 
 15  winner_age          110386 non-nul

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,1990-339,Adelaide,Hard,32,A,19900101,1,101843,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,591.0,282.0,70.0
1,1,1990-339,Adelaide,Hard,32,A,19900101,2,100714,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,146.0,157.0,127.0,182.0
2,2,1990-339,Adelaide,Hard,32,A,19900101,3,101529,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,113.0,215.0,71.0,290.0
3,3,1990-339,Adelaide,Hard,32,A,19900101,4,100628,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,297.0,80.0,271.0
4,4,1990-339,Adelaide,Hard,32,A,19900101,5,101404,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,34.0,496.0,100.0,224.0


To start there are data errors for certain variables:

1. tourney_id - The '-' and inclusion of exhibition tournaments (Davis Cup) needs to be formatted to set datatype from object() to int()


In [140]:
# Removing Davis Cup Tournaments
df = df[~df['tourney_id'].str.contains('[a-zA-Z]', regex=True, na=False)]

# Converting tourney_id to int64()
df['tourney_id'] = df['tourney_id'].str.replace('-', '').astype(int)

df.head()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,1990339,Adelaide,Hard,32,A,19900101,1,101843,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,591.0,282.0,70.0
1,1,1990339,Adelaide,Hard,32,A,19900101,2,100714,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,146.0,157.0,127.0,182.0
2,2,1990339,Adelaide,Hard,32,A,19900101,3,101529,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,113.0,215.0,71.0,290.0
3,3,1990339,Adelaide,Hard,32,A,19900101,4,100628,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,297.0,80.0,271.0
4,4,1990339,Adelaide,Hard,32,A,19900101,5,101404,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,34.0,496.0,100.0,224.0


### Tournament Entry and Seeding Formats
Tournament Entry and Main Draw Composition:
ATP Tour events typically have a set draw size—commonly 28, 32, 48, 56, or 64 players in singles, depending on the tournament category (e.g., ATP 250, ATP 500, Masters 1000, or Grand Slam events, which are not ATP-run but follow similar principles). The main draw field usually consists of:

1. **Direct Acceptances**: Players who are admitted based on their ATP ranking at a specified cutoff date. The top-ranked players sign up in advance, and if their ranking is high enough, they enter the main draw without having to qualify.

2. **Wild Cards**: These are spots awarded at the discretion of the tournament organizers. Wild cards enable local players, promising juniors, returning former champions, or players who have been injured and do not have a high enough ranking, to enter directly into the main draw. They bypass the normal entry criteria, adding interest and diversity to the field.

3. **Qualifiers**: A small number of players earn their place in the main draw by playing in a qualification tournament held just prior to the main event. Qualifying draws might consist of multiple rounds where lower-ranked or unranked players compete, and a handful of winners from that mini-tournament get "Q" spots in the main draw.

4. **Special Exempt and Protected Ranking Entries**: Under specific circumstances, a player may enter using a protected ranking (if returning from a long injury break) or via a special exemption if their previous tournament’s deep run prevented them from participating in that event’s qualifying.

#### Seeding

Once the main draw field is finalized, the highest-ranked players are given seedings. The number of seeds typically corresponds to a fraction of the draw (e.g., in a 32-draw, you might have 8 seeds; in a 64-draw, 16 seeds).

#### Unseeded Random Draw

After placing the seeds, the remaining players (those who are neither seeded nor wild cards specifically allocated to certain draw positions) are randomly drawn and placed into the remaining slots.

### Variable Reduction

A lot of the match based variables should not contribute to 'opportunity'. For example, a point won through an ace compared to a regular rally-based point won has no measurable difference in the context of opportunity.

Using the same logic, we can say it doesn't matter how a player win's their match, the binary outcome of a match can be seen in two stages:
1. Presented Opportunity - The random draw through which the players are assigned an opponent. This 'exogenous shock' can be measured as the rank differential faced by players in the random draw stage of the tournament.

2. Opportunity Outcome - If the upset does occur, then a 'formidable opponent' has been removed from the player's tournament path. This also could also be seen as making the tournament easier for the unseeded players that are in the same tournament branch as the upset.

**Note** - Is it possible to use rank difference as an instrument? Since the initial draw is random?


In [141]:
df.head()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,1990339,Adelaide,Hard,32,A,19900101,1,101843,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,591.0,282.0,70.0
1,1,1990339,Adelaide,Hard,32,A,19900101,2,100714,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,146.0,157.0,127.0,182.0
2,2,1990339,Adelaide,Hard,32,A,19900101,3,101529,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,113.0,215.0,71.0,290.0
3,3,1990339,Adelaide,Hard,32,A,19900101,4,100628,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,297.0,80.0,271.0
4,4,1990339,Adelaide,Hard,32,A,19900101,5,101404,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,34.0,496.0,100.0,224.0


Based on this, We can drop the following vars:

w_ace
- winner's number of aces
w_df
- winner's number of doubles faults
w_svpt
- winner's number of serve points
w_1stIn
- winner's number of first serves made
w_1stWon
- winner's number of first-serve points won
w_2ndWon
- winner's number of second-serve points won
w_SvGms
- winner's number of serve games
w_bpSaved
- winner's number of break points saved
w_bpFaced
- winner's number of break points faced

duplicate metrics for loser
- l_ace
- l_df
- l_svpt
- l_1stIn
- l_1stWon
- l_2ndWon
- l_SvGms
- l_bpSaved
- l_bpFaced

In [142]:
cols_to_drop = [
    "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced",
    "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced", "Unnamed: 0"
]

df = df.drop(columns=cols_to_drop, errors='ignore')
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,loser_ioc,loser_age,score,best_of,round,minutes,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1990339,Adelaide,Hard,32,A,19900101,1,101843,1.0,NaN,...,SWE,20.3,7-6 6-2,3,R32,NaN,25.0,591.0,282.0,70.0
1,1990339,Adelaide,Hard,32,A,19900101,2,100714,NaN,WC,...,RSA,25.0,6-4 6-4,3,R32,NaN,146.0,157.0,127.0,182.0
2,1990339,Adelaide,Hard,32,A,19900101,3,101529,NaN,NaN,...,SWE,23.9,6-4 6-3,3,R32,NaN,113.0,215.0,71.0,290.0
3,1990339,Adelaide,Hard,32,A,19900101,4,100628,7.0,NaN,...,CZE,21.9,6-3 6-4,3,R32,NaN,68.0,297.0,80.0,271.0
4,1990339,Adelaide,Hard,32,A,19900101,5,101404,3.0,NaN,...,ITA,22.3,6-4 4-6 6-3,3,R32,NaN,34.0,496.0,100.0,224.0


## Creating the Panel

In [143]:
# Convert tournry_date to Year
df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df['year'] = df['tourney_date'].dt.year

Now, reshape the data to Player x Matches. We'll create a dataset for winner and losers.

In [144]:
winners = df[['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'year',
              'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'winner_name', 'winner_hand', 'winner_ht',
              'winner_ioc', 'winner_age', 'score', 'best_of', 'round', 'minutes', 'winner_rank', 'winner_rank_points']].copy()

losers = df[['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'year',
             'match_num', 'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand', 'loser_ht',
             'loser_ioc', 'loser_age', 'score', 'best_of', 'round', 'minutes', 'loser_rank', 'loser_rank_points']].copy()

In [145]:
winners = winners.rename(columns={
    'winner_id': 'player_id',
    'winner_seed': 'player_seed',
    'winner_entry': 'player_entry',
    'winner_name': 'player_name',
    'winner_hand': 'player_hand',
    'winner_ht': 'player_ht',
    'winner_ioc': 'player_ioc',
    'winner_age': 'player_age',
    'winner_rank': 'player_rank',
    'winner_rank_points': 'player_rank_points'
})

losers = losers.rename(columns={
    'loser_id': 'player_id',
    'loser_seed': 'player_seed',
    'loser_entry': 'player_entry',
    'loser_name': 'player_name',
    'loser_hand': 'player_hand',
    'loser_ht': 'player_ht',
    'loser_ioc': 'player_ioc',
    'loser_age': 'player_age',
    'loser_rank': 'player_rank',
    'loser_rank_points': 'player_rank_points'
})

In [146]:
# Add a variable to indicate match outcome for the player
winners['match_outcome'] = 'win'
losers['match_outcome'] = 'loss'

In [147]:
losers.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,player_id,player_seed,...,player_ht,player_ioc,player_age,score,best_of,round,minutes,player_rank,player_rank_points,match_outcome
0,1990339,Adelaide,Hard,32,A,1990-01-01,1990,1,101632,NaN,...,190.0,SWE,20.3,7-6 6-2,3,R32,NaN,282.0,70.0,loss
1,1990339,Adelaide,Hard,32,A,1990-01-01,1990,2,101096,NaN,...,190.0,RSA,25.0,6-4 6-4,3,R32,NaN,127.0,182.0,loss
2,1990339,Adelaide,Hard,32,A,1990-01-01,1990,3,101232,NaN,...,183.0,SWE,23.9,6-4 6-3,3,R32,NaN,71.0,290.0,loss
3,1990339,Adelaide,Hard,32,A,1990-01-01,1990,4,101434,NaN,...,190.0,CZE,21.9,6-3 6-4,3,R32,NaN,80.0,271.0,loss
4,1990339,Adelaide,Hard,32,A,1990-01-01,1990,5,101387,NaN,...,180.0,ITA,22.3,6-4 4-6 6-3,3,R32,NaN,100.0,224.0,loss


In [148]:
winners.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,player_id,player_seed,...,player_ht,player_ioc,player_age,score,best_of,round,minutes,player_rank,player_rank_points,match_outcome
0,1990339,Adelaide,Hard,32,A,1990-01-01,1990,1,101843,1.0,...,188.0,ESP,18.9,7-6 6-2,3,R32,NaN,25.0,591.0,win
1,1990339,Adelaide,Hard,32,A,1990-01-01,1990,2,100714,NaN,...,183.0,AUS,29.0,6-4 6-4,3,R32,NaN,146.0,157.0,win
2,1990339,Adelaide,Hard,32,A,1990-01-01,1990,3,101529,NaN,...,193.0,GER,21.2,6-4 6-3,3,R32,NaN,113.0,215.0,win
3,1990339,Adelaide,Hard,32,A,1990-01-01,1990,4,100628,7.0,...,201.0,NED,30.2,6-3 6-4,3,R32,NaN,68.0,297.0,win
4,1990339,Adelaide,Hard,32,A,1990-01-01,1990,5,101404,3.0,...,180.0,AUT,22.2,6-4 4-6 6-3,3,R32,NaN,34.0,496.0,win


In [149]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,loser_age,score,best_of,round,minutes,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
0,1990339,Adelaide,Hard,32,A,1990-01-01,1,101843,1.0,NaN,...,20.3,7-6 6-2,3,R32,NaN,25.0,591.0,282.0,70.0,1990
1,1990339,Adelaide,Hard,32,A,1990-01-01,2,100714,NaN,WC,...,25.0,6-4 6-4,3,R32,NaN,146.0,157.0,127.0,182.0,1990
2,1990339,Adelaide,Hard,32,A,1990-01-01,3,101529,NaN,NaN,...,23.9,6-4 6-3,3,R32,NaN,113.0,215.0,71.0,290.0,1990
3,1990339,Adelaide,Hard,32,A,1990-01-01,4,100628,7.0,NaN,...,21.9,6-3 6-4,3,R32,NaN,68.0,297.0,80.0,271.0,1990
4,1990339,Adelaide,Hard,32,A,1990-01-01,5,101404,3.0,NaN,...,22.3,6-4 4-6 6-3,3,R32,NaN,34.0,496.0,100.0,224.0,1990


Comparing the winner and loser dataframe to the master data set shows that winners and losers have been seperated properly. **(Sergi Bruguera vs Per Henricsson)**

Now we can concatenate the winner and loser dataframes

In [150]:
player_match_df = pd.concat([winners, losers], ignore_index=True)

In [151]:
# Now, we have one row per player per match. To get the opponent's rank, we do a self-merge.
# First, isolate identifying columns and player_rank:
match_players = player_match_df[['tourney_id', 'match_num', 'player_id', 'player_rank']]

# Merge player_match_df with match_players on tourney_id and match_num
player_match_df = player_match_df.merge(
    match_players,
    on=['tourney_id', 'match_num'],
    suffixes=('', '_opp')
)

# Now each row is merged with both player's ranks. We only want the opponent row:
player_match_df = player_match_df[player_match_df['player_id_opp'] != player_match_df['player_id']]

# Rename the opponent's player_rank column
player_match_df = player_match_df.rename(columns={'player_rank_opp': 'opponent_rank'})


Now, we need to re-label the rounds

In [152]:
player_match_df["round"].unique()

array(['R32', 'R16', 'QF', 'SF', 'F', 'R128', 'R64', 'RR', 'BR', 'ER'],
      dtype=object)

In [154]:
round_order = {
    'R128': 1,
    'R64': 2,
    'R32': 3,
    'R16': 4,
    'QF': 5,
    'SF': 6,
    'F': 7,
    'RR': 8,
    'BR': 9,  # Bonus round or playoff
    'ER': 10  # Extra round if applicable
}

In [155]:
player_match_df['round_num'] = player_match_df['round'].map(round_order)

In [156]:
# Sort by year, tournament, player, and match number to get the order of matches for each player
player_match_df = player_match_df.sort_values(by=['player_id', 'year', 'tourney_id', 'match_num'])

In [157]:
# Let's also capture the rank of the opponent. Since currently each row is from the perspective of one player,
# we can identify the opponent's rank. For a 'win' row, the opponent is the loser, for a 'loss' row, the opponent is the winner.

# Create a column 'opponent_rank' by merging player-match data on itself
# For simplicity, we can first separate winners and losers again and merge:
winners_info = player_match_df[player_match_df['match_outcome'] == 'win'][['tourney_id', 'match_num', 'player_id', 'player_rank']]
losers_info = player_match_df[player_match_df['match_outcome'] == 'loss'][['tourney_id', 'match_num', 'player_id', 'player_rank']]


In [158]:
# Merge winners with their losing opponents (based on tourney_id and match_num)
match_opponents = player_match_df[['tourney_id', 'match_num', 'player_id', 'match_outcome']]
match_opponents = match_opponents.merge(
    winners_info.rename(columns={'player_id':'winner_pid','player_rank':'winner_rank'}),
    on=['tourney_id','match_num'], how='left'
)

In [159]:
match_opponents = match_opponents.merge(
    losers_info.rename(columns={'player_id':'loser_pid','player_rank':'loser_rank'}),
    on=['tourney_id','match_num'], how='left'
)

In [160]:
# Assign opponent rank depending on outcome
match_opponents['opponent_rank'] = match_opponents.apply(
    lambda x: x['loser_rank'] if x['match_outcome'] == 'win' else x['winner_rank'], axis=1
)

In [161]:
# Merge 'opponent_rank' back into player_match_df
player_match_df = player_match_df.merge(
    match_opponents[['tourney_id','match_num','player_id','opponent_rank']],
    on=['tourney_id','match_num','player_id'], how='left'
)

In [162]:
player_match_df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,player_id,player_seed,...,best_of,round,minutes,player_rank,player_rank_points,match_outcome,player_id_opp,opponent_rank_x,round_num,opponent_rank_y
0,1992323,Bordeaux,Clay,32,A,1992-09-14,1992,7,100282,NaN,...,3,R32,109.0,410.0,47.0,loss,102001,99.0,3,99.0
1,1992409,Atlanta,Clay,32,A,1992-04-27,1992,12,100282,NaN,...,3,R32,167.0,444.0,38.0,loss,101334,18.0,3,18.0
2,1990327,Toulouse,Hard,32,A,1990-10-01,1990,8,100284,NaN,...,3,R32,NaN,125.0,282.0,loss,101086,30.0,3,30.0
3,1990328,Basel,Hard,32,A,1990-09-24,1990,14,100284,NaN,...,3,R32,NaN,121.0,281.0,loss,101529,65.0,3,65.0
4,1990408,Milan,Carpet,32,A,1990-02-05,1990,8,100284,6.0,...,3,R32,NaN,19.0,NaN,loss,101482,134.0,3,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195489,20240403,Miami Masters,Hard,128,M,2024-03-18,2024,266,212021,NaN,...,3,R64,75.0,360.0,137.0,loss,210097,17.0,2,17.0
195490,20240425,Barcelona,Clay,64,A,2024-04-15,2024,240,212021,NaN,...,3,R64,140.0,328.0,160.0,loss,200572,229.0,2,229.0
195491,20241536,Madrid Masters,Clay,128,M,2024-04-22,2024,203,212021,NaN,...,3,R128,94.0,322.0,160.0,loss,127157,61.0,1,61.0
195492,20230747,Beijing,Hard,32,A,2023-09-27,2023,276,212044,NaN,...,3,R32,70.0,905.0,18.0,loss,200221,25.0,3,25.0


In [163]:
player_match_df = player_match_df.drop(columns=['opponent_rank_y'])
player_match_df = player_match_df.rename(columns={'opponent_rank_x': 'opp_rank'})
player_match_df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,player_id,player_seed,...,score,best_of,round,minutes,player_rank,player_rank_points,match_outcome,player_id_opp,opp_rank,round_num
0,1992323,Bordeaux,Clay,32,A,1992-09-14,1992,7,100282,NaN,...,6-4 3-6 6-0,3,R32,109.0,410.0,47.0,loss,102001,99.0,3
1,1992409,Atlanta,Clay,32,A,1992-04-27,1992,12,100282,NaN,...,5-7 7-6(6) 6-3,3,R32,167.0,444.0,38.0,loss,101334,18.0,3
2,1990327,Toulouse,Hard,32,A,1990-10-01,1990,8,100284,NaN,...,6-4 6-4,3,R32,NaN,125.0,282.0,loss,101086,30.0,3
3,1990328,Basel,Hard,32,A,1990-09-24,1990,14,100284,NaN,...,7-6 6-3,3,R32,NaN,121.0,281.0,loss,101529,65.0,3
4,1990408,Milan,Carpet,32,A,1990-02-05,1990,8,100284,6.0,...,6-3 4-6 7-6,3,R32,NaN,19.0,NaN,loss,101482,134.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195489,20240403,Miami Masters,Hard,128,M,2024-03-18,2024,266,212021,NaN,...,6-3 6-4,3,R64,75.0,360.0,137.0,loss,210097,17.0,2
195490,20240425,Barcelona,Clay,64,A,2024-04-15,2024,240,212021,NaN,...,2-6 6-4 6-3,3,R64,140.0,328.0,160.0,loss,200572,229.0,2
195491,20241536,Madrid Masters,Clay,128,M,2024-04-22,2024,203,212021,NaN,...,6-1 7-5,3,R128,94.0,322.0,160.0,loss,127157,61.0,1
195492,20230747,Beijing,Hard,32,A,2023-09-27,2023,276,212044,NaN,...,6-2 6-2,3,R32,70.0,905.0,18.0,loss,200221,25.0,3


In [164]:
first_four = ['player_id', 'year', 'tourney_id', 'match_num']

# Reorder columns so the specified ones come first, followed by the rest
player_match_df = player_match_df[first_four + [col for col in player_match_df.columns if col not in first_four]]

player_match_df = player_match_df.sort_values(by=['player_id', 'year', 'tourney_id', 'match_num'])

player_match_df.columns

Index(['player_id', 'year', 'tourney_id', 'match_num', 'tourney_name',
       'surface', 'draw_size', 'tourney_level', 'tourney_date', 'player_seed',
       'player_entry', 'player_name', 'player_hand', 'player_ht', 'player_ioc',
       'player_age', 'score', 'best_of', 'round', 'minutes', 'player_rank',
       'player_rank_points', 'match_outcome', 'player_id_opp', 'opp_rank',
       'round_num'],
      dtype='object')

### Identifying First/Second Round opponent rank

In [173]:
# This function capture the first-round opponent's rank
def first_round_opponent_rank(group):
    # Filter only first match played by player in that tournament
    first_match = group[group['match_num'] == group['match_num'].min()]
    if len(first_match) > 0:
        return first_match['opp_rank'].iloc[0]
    else:
        return None

In [174]:
# This function capture the average of the first two rounds of the opponent's rank
def avg_first_two_opponents_rank(group):
    # Take the first two matches of the tournament for this player
    first_two_matches = group.nsmallest(2, 'match_num')
    if len(first_two_matches) > 0:
        return first_two_matches['opp_rank'].mean()
    else:
        return None

In [175]:
# Use .apply() on the groupby object
player_tourney_panel = (
    player_match_df
    .groupby(['player_id', 'year', 'tourney_id'], as_index=False)
    .apply(lambda g: pd.Series({
        'player_name': g['player_name'].iloc[0],
        'player_seed': g['player_seed'].iloc[0],
        'player_rank': g['player_rank'].iloc[0],
        'player_age': g['player_age'].iloc[0],
        'tourney_name': g['tourney_name'].iloc[0],
        'tourney_level': g['tourney_level'].iloc[0],
        'surface': g['surface'].iloc[0],
        'max_round_num': g['round_num'].max(),   # max round reached
        'matches_played': g['match_num'].count(), # matches played
        'first_round_opp_rank': first_round_opponent_rank(g),
        'avg_first_two_opp_rank': avg_first_two_opponents_rank(g)
    }))
)

player_tourney_panel = player_tourney_panel.reset_index(drop=True)

In [177]:
player_tourney_panel
player_tourney_panel.to_csv(r"C:\Users\shivg\OneDrive\Desktop\GitHub\Tennis_ATP\data\Player_Year_Tourney_Panel.csv")